Research Track 2 Assignment developed by Yusuke Kido (5239225)
=====


# How to execute

You should run the launching file below.
```
$ roslaunch rt2_first_assignment rt2_first_assignment.launch
```
After launching, you should run Restart & Run All' command in jupyter notebook and you can manipulate the car robot.

## Import libralies

In [1]:
import rospy
import time
import os
import tf
import sys
import actionlib
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets as widgets
import jupyros as jr
from nav_msgs.msg import Odometry
from tf.transformations import quaternion_matrix
from matplotlib.animation import FuncAnimation
from matplotlib import pyplot as plt
from jupyros import ros3d
from geometry_msgs.msg import Twist
from sensor_msgs.msg import LaserScan
from IPython.display import display
from IPython.display import clear_output
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
from std_msgs.msg import Float32, Int32
from rt2_first_assignment.srv import Cmd
from rt2_first_assignment.srv import Dist
from rt2_first_assignment.msg import reach

## Initializing ROS node

In [2]:
rospy.init_node('jupy')
# Service for selecting the mode
client_cmd = rospy.ServiceProxy("/cmd", Cmd)
# Service for the destination in auto drive mode
client_dist = rospy.ServiceProxy("/dist", Dist)
# publishing velocity 
vel_pub = jr.publish('cmd_vel', Twist)

## UI: implementing the buttons

In [3]:
# buttons for selecting modes
b1 = Button(description = 'Auto_drive mode',
           layout = Layout(width = 'auto', align = "center", grid_area = 'b1'),
           style = ButtonStyle(button_color = 'GreenYellow'))

b2 = Button(description = 'Manual_drive mode',
           layout = Layout(width = 'auto', grid_area = 'b2'),
           style = ButtonStyle(button_color = 'GreenYellow'))

b3 = Button(description = 'Assisted_drive mode',
           layout = Layout(width = 'auto', grid_area = 'b3'),
           style = ButtonStyle(button_color = 'GreenYellow'))

b4 = Button(description = 'Cancel Goal',
           layout = Layout(width = 'auto', grid_area = 'b4'),
           style = ButtonStyle(button_color = 'Yellow'))

b5 = Button(description = 'Reset Position',
           layout = Layout(width = 'auto', grid_area = 'b5'),
           style = ButtonStyle(button_color = 'lightblue'))

b6 = Button(description = 'End Program',
           layout = Layout(width = 'auto', grid_area = 'b6'),
           style = ButtonStyle(button_color = 'red'))

b7 = Button(description = 'Send Goal',
           layout = Layout(width = 'auto', grid_area = 'b7'),
           style = ButtonStyle(button_color = 'GreenYellow'))


# buttons for manual mode
b8 = Button(description = 'Turn Left',
           layout = Layout(width = 'auto', grid_area = 'b8'),
           style = ButtonStyle(button_color = 'GreenYellow'))

b9 = Button(description = 'Move Forward',
           layout = Layout(width = 'auto', grid_area = 'b9'),
           style = ButtonStyle(button_color = 'GreenYellow'))

b10 = Button(description = 'Turn Right',
           layout = Layout(width = 'auto', grid_area = 'b10'),
           style = ButtonStyle(button_color = 'GreenYellow'))

b11 = Button(description = 'Move Backward',
           layout = Layout(width = 'auto', grid_area = 'b11'),
           style = ButtonStyle(button_color = 'GreenYellow'))

b12 = Button(description = 'Stop',
           layout = Layout(width = 'auto', grid_area = 'b12'),
           style = ButtonStyle(button_color = 'Yellow'))


# getting the destination in auto drive mode
decided_x = widgets.FloatText(
    value = 0,
    description = 'X goal:',
    disabled = False,
)

decided_y = widgets.FloatText(
    value = 0,
    description = 'Y goal:',
    disabled = False,
)

## Function related to robot movement

In [4]:
# Define vel msg
vel_msg = Twist()
# Publisher for the velocity  in　Manual Drive　and assisted_drive.
input_vel_pub = rospy.Publisher('/input_cmd_vel', Twist, queue_size = 100)
global cmd, pos_x, pos_y, vel_msg, vel
    
# the fuctions in each mode
def auto_drive(b):
    rospy.wait_for_service("/cmd")
    print("Auto Drive")
    cmd = 1
    res1 = client_cmd(cmd)

def manual_drive(b):
    rospy.wait_for_service("/cmd")
    print("Manual Drive")
    cmd = 2
    res1 = client_cmd(cmd)
     
def assisted_drive(b):
    rospy.wait_for_service("/cmd")
    print("Assisted Drive")
    cmd = 3
    res1 = client_cmd(cmd)
    
def cancel_goal(b):
    rospy.wait_for_service("/cmd")
    print("cancel the goal")
    cmd = 4
    res1 = client_cmd(cmd)
    
def reset_position(b):
    rospy.wait_for_service("/cmd")
    print("car's position is resetted")
    cmd = 5
    res1 = client_cmd(cmd)
    
def end_program(b):
    rospy.wait_for_service("/cmd")
    print("end program")
    cmd = 0
    res1 = client_cmd(cmd)
    
# the destenation is fixed in auto drive mode
def set_position(b):
    rospy.wait_for_service("/dist")
    global decided_x, decided_y
    pos_x = decided_x.value
    pos_y = decided_y.value
    res2 = client_dist(pos_x, pos_y)
    print("destination is sent")
    
# Publishes the velocity in　Manual Drive　and assisted_drive.
def move_forward(b):
    vel_msg.linear.x = 0.5
    vel_msg.linear.y = 0
    vel_msg.linear.z = 0
    vel_msg.angular.x = 0
    vel_msg.angular.y = 0
    vel_msg.angular.z = 0
    input_vel_pub.publish(vel_msg)

def turn_left(b):
    vel_msg.linear.x = 0
    vel_msg.linear.y = 0
    vel_msg.linear.z = 0
    vel_msg.angular.x = 0
    vel_msg.angular.y = 0
    vel_msg.angular.z = 0.5
    input_vel_pub.publish(vel_msg)

def turn_right(b):
    vel_msg.linear.x = 0
    vel_msg.linear.y = 0
    vel_msg.linear.z = 0
    vel_msg.angular.x = 0
    vel_msg.angular.y = 0
    vel_msg.angular.z = -0.5
    input_vel_pub.publish(vel_msg)
    
def move_backward(b):
    vel_msg.linear.x = -0.5
    vel_msg.linear.y = 0
    vel_msg.linear.z = 0
    vel_msg.angular.x = 0
    vel_msg.angular.y = 0
    vel_msg.angular.z = 0
    input_vel_pub.publish(vel_msg)
    
def stop(b):
    vel_msg.linear.x = 0
    vel_msg.linear.y = 0
    vel_msg.linear.z = 0
    vel_msg.angular.x = 0
    vel_msg.angular.y = 0
    vel_msg.angular.z = 0
    input_vel_pub.publish(vel_msg)
    
# each functions is called when a button is clicked
b1.on_click(auto_drive)
b2.on_click(manual_drive)
b3.on_click(assisted_drive)
b4.on_click(cancel_goal)
b5.on_click(reset_position)
b6.on_click(end_program)
b7.on_click(set_position)
b8.on_click(turn_left)
b9.on_click(move_forward)
b10.on_click(turn_right)
b11.on_click(move_backward)
b12.on_click(stop)

## Choose the mode

In [5]:
print('Choose the mode')

GridBox(children = [b1, b2, b3, b4, b5, b6],
       layout = Layout(
           width = '100%',
           grid_template_rows = 'auto auto auto',
           grid_template_columns = '50% 50%',
           grid_template_areas = '''
           "b1 b4"
           "b2 b5"
           "b3 b6"
           ''')
        )

Choose the mode


GridBox(children=(Button(description='Auto_drive mode', layout=Layout(grid_area='b1', width='auto'), style=But…

Auto Drive
Manual Drive
cancel the goal
car's position is resetted
Auto Drive
Auto Drive
car's position is resetted
Assisted Drive
Manual Drive
car's position is resetted
Auto Drive
cancel the goal
car's position is resetted
Manual Drive
end program


## Select (x,y) goal in auto drive mode

In [6]:
print('Select (x,y) goal position and send the goal ')

GridBox(children = [decided_x, decided_y, b7],
       layout = Layout(
           width = '100%',
           grid_template_rows = 'auto auto',
           grid_template_columns = '50% 50%',
           grid_template_areas = '''
           "decided_x decided_y"
           "b7 b7"
           ''')
        )

Select (x,y) goal position and send the goal 


GridBox(children=(FloatText(value=0.0, description='X goal:'), FloatText(value=0.0, description='Y goal:'), Bu…

destination is sent
destination is sent
destination is sent
destination is sent


## Choose direction in manual mode and assisted mode

In [7]:
print('Maneuver the robot')

GridBox(children = [b8, b9, b10, b11, b12],
       layout = Layout(
           width = '100%',
           grid_template_rows = 'auto auto auto',
           grid_template_columns = '33% 33% 33%',
           grid_template_areas = '''
           "b9 b9 b9"
           "b8 b12 b10"
           "b11 b11 b11"
           ''')
        )

Maneuver the robot


GridBox(children=(Button(description='Turn Left', layout=Layout(grid_area='b8', width='auto'), style=ButtonSty…

## Data visualization for robot's position and whether it reaches or not

In [19]:
class Odom_Visualiser:
    def __init__(self):
        self.fig_odom, self.ax1 = plt.subplots()
        plt.grid(True)
        self.ln1, = self.ax1.plot([], [], 'ro')
        self.x_data, self.y_data = [], []
    def plot_init(self):
        self.ax1.set_title("Robot Odometry", fontsize = 20, fontweight = 'bold')
        self.ax1.set_xlabel("X [m]", fontsize = 10, fontweight = "bold")
        self.ax1.set_ylabel("Y [m]", fontsize = 10, fontweight = "bold")
        self.ax1.set_xlim(-20, 20)
        self.ax1.set_ylim(-20, 20)
        return self.ln1
    
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)         
        
    def update_plot(self, frame):
        self.ln1.set_data(self.x_data, self.y_data)
        return self.ln1

class Laser_Visualiser:
    def __init__(self):
        self.fig_laser = plt.figure(figsize = (7, 7))
        self.ax2 = plt.subplot(111, polar = True)
        plt.grid(True)
        self.ax2.set_thetalim(-np.pi/2, np.pi/2)
        self.ax2.set_rmax(20)
        self.ax2.set_theta_zero_location("N")
        self.ln2, = self.ax2.plot([], [], 'b-')
        self.laser_dist = []
        self.laser_ang = []
        
    def plot_init(self):
        self.ax2.set_title("Laser Scanner Data", fontsize = 20, fontweight = "bold")
        self.ax2.set_xlabel("Distance [m]", fontsize = 10, fontweight = "bold")
        return self.ln2
    
    def laser_callback(self, msg):
        global laser_ang
        self.laser_dist = msg.ranges
        self.laser_ang = np.arange(msg.angle_min, msg.angle_max+msg.angle_increment, msg.angle_increment)
        
    def update_plot(self, frame):
        global laser_ang
        self.ax2.set_rmax(20)
        self.ln2.set_data(self.laser_ang, self.laser_dist)
        return self.ln2
    
class Goal_Visualiser:
    def __init__(self):
        self.fig_goal, self.ax3 = plt.subplots()
        plt.grid(False)
        self.reached = 0
        self.unreached = 0
        self.ln3 = plt.bar(['Reached', 'Unreached'], [self.reached, self.unreached], color = ['red', 'green'])
        
    def plot_init(self):
        self.ax3.set_title("Goals are reached or not", fontsize = 20, fontweight = "bold")
        self.ax3.set_xlabel("Goal success", fontsize = 10, fontweight = "bold")
        self.ax3.set_ylabel("Numbers", fontsize = 10, fontweight = "bold")
        self.ax3.set_ylim(0, 16)
        self.ax3.set_yticks(np.arange(0, 16, 2))
        return self.ln3
    
    def goal_callback(self, msg):
        self.reached = msg.reach
        self.unreached = msg.unreach     
        
    def update_plot(self, frame):
        self.ln3 = plt.bar(['Reached', 'Unreached'], [self.reached, self.unreached], color = ['red', 'green'])
        return self.ln3

In [21]:
%matplotlib widget
odom_vis = Odom_Visualiser()
subOdom = rospy.Subscriber('/odom', Odometry, odom_vis.odom_callback)
ani1 = FuncAnimation(odom_vis.fig_odom, odom_vis.update_plot, init_func=odom_vis.plot_init,interval=1000, blit=False)
laser_vis = Laser_Visualiser()
subLaser = rospy.Subscriber('/scan', LaserScan, laser_vis.laser_callback)
ani2 = FuncAnimation(laser_vis.fig_laser, laser_vis.update_plot, init_func=laser_vis.plot_init,interval=500, blit=False)
goal_vis = Goal_Visualiser()
subGoal = rospy.Subscriber('/status_goal', reach, goal_vis.goal_callback)
ani3= FuncAnimation(goal_vis.fig_goal, goal_vis.update_plot, init_func=goal_vis.plot_init,interval=500, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …